# Preprocesamiento

In [85]:
import pandas as pd

import sys
sys.path.append("..")

from src.support_prep import *
from src.support_models import *    

import pickle

In [86]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [87]:
df = pd.read_pickle("../datos/clean.pkl")

In [88]:
df.head()

,Age,Attrition,BusinessTravel,Department,DistanceFromHome,Education,EducationField,EmployeeID,Gender,JobLevel,...,TotalWorkingYears,TrainingTimesLastYear,YearsAtCompany,YearsSinceLastPromotion,YearsWithCurrManager,EnvironmentSatisfaction,JobSatisfaction,WorkLifeBalance,JobInvolvement,PerformanceRating
0,51,No,Travel_Rarely,Sales,6,2,Life Sciences,1,Female,1,...,1,6,1,0,0,3.0,4,2,3,3
1,31,Yes,Travel_Frequently,Research & Development,10,1,Life Sciences,2,Female,1,...,6,3,5,1,4,3.0,2,4,2,4
2,32,No,Travel_Frequently,Research & Development,17,4,Other,3,Male,4,...,5,2,5,0,3,2.0,2,1,3,3
3,38,No,Non-Travel,Research & Development,2,5,Life Sciences,4,Male,3,...,13,5,8,7,5,4.0,4,3,2,3
4,32,No,Travel_Rarely,Research & Development,10,1,Medical,5,Male,1,...,9,2,6,0,4,4.0,1,3,3,3


## Encoding

Para realizar el encoding, vamos a necesitar convertir nuestra variable respuesta en binario.

In [89]:
df["Attrition"] = df["Attrition"].map({"No":0, "Yes":1})

Ahora separamos nuestras variables numéricas de las categóricas, además de que para el preprocesamiento no querremos tocar la variable respuesta, y nos quitamos también el EmployeeID, ya que es esencialmente un índice que no aporta información.

In [90]:
cat_cols = df.select_dtypes("O").columns
num_cols = df.select_dtypes("number").columns.drop(["Attrition", "EmployeeID"])

In [91]:
df = df.drop(columns="EmployeeID")

Con una función del src podemos obtener aquellas categorías que sí presentan un orden, es decir, las proporciones de sus matrices de contingencia no coinciden con las observaciones esperadas, por lo que hay categorías con mayor relevancia que otras.

In [92]:
cat_diff = detectar_orden_cat(df = df, lista_cat=cat_cols,var_respuesta="Attrition", show=False)

In [93]:
cat_diff

['BusinessTravel',
 'Department',
 'EducationField',
 'JobRole',
 'MaritalStatus',
 'EnvironmentSatisfaction',
 'JobSatisfaction',
 'WorkLifeBalance',
 'JobInvolvement']

In [94]:
cat_no_diff = cat_cols.drop(cat_diff)

In [95]:
cat_no_diff

Index(['Education', 'Gender', 'JobLevel', 'StockOptionLevel',
       'PerformanceRating'],
      dtype='object')

Vemos que las únicas sin orden específico son el nivel educativo, el género, el nivel del puesto dentro de la empresa, las opciones de acciones y la valoración de desempeño.

Vamos a hacer un target encoding para las categorías con diferencias y un onehot encoding con las categorías sin diferencias.

In [96]:
df_target, target = encode_target(data = df, columns = cat_diff, response_var="Attrition")

In [97]:
df_oh, onehot = encode_onehot(df, columns = cat_no_diff)

In [98]:
df = pd.concat([df["Attrition"], df_target.drop(columns=cat_no_diff), df_oh], axis = 1)

Guardamos los objetos generados en pkl para poder reutilizar los encodings.

In [99]:
with open('../models/onehot.pkl', 'wb') as file:
    pickle.dump(onehot, file)
with open('../models/target.pkl', 'wb') as file:
    pickle.dump(target, file)

## Feature Scaling

Ahora realizaremos una estandarización para nuestras variables, para que nuestro modelo no esté sesgado por ciertas variables que pudieran tener escalas mayores a otras, como el salario sobre los años trabajados. 

Se escoge minmax, ya que por una parte tenemos pocos outliers univariantes y por otra parte, al ser nuestra variable respuesta binaria, el target encoder nos genera valores entre 0 y 1 (representan la probabilidad de Attrition según las características de cada categoría), y el onehot también nos limita a 0 y 1, por lo que la escala sería adecuada.

In [100]:
df_scaled, scaler = scale_data(data = df, columns=df.columns.drop(df_oh.columns).drop("Attrition"), method = "minmax")

In [101]:
df[df_scaled.columns] = df_scaled

También guardamos este objeto en un pkl.

In [102]:
with open('../models/scaler.pkl', 'wb') as file:
    pickle.dump(scaler, file)

## Outliers

Por último vamos a encontrar y tratar los outliers en nuestros registros. Seguiremos una estrategia de dos pasos. De primeras buscaremos aquellos outliers más específicos, que podrían representar valores erróneos o muy atípicos. Para ello usaremos una estrategia de Isolation Forest.

In [103]:
df_outliers_ifo, ifo = find_outliers(data = df.drop(columns = "Attrition"), columns = df.drop(columns = "Attrition").columns, method = "ifo")

100%|██████████| 25/25 [01:03<00:00,  2.54s/it]


In [104]:
df_outliers_ifo.shape[0]/df.shape[0]

0.02018140589569161

Dado que encontramos una proporción pequeña de outliers, decidimos eliminar estos registros, ya que podrían generar ruido posteriormente en el entrenamiento del modelo.

In [105]:
df.drop(index=df_outliers_ifo.index, inplace=True)

In [106]:
df.reset_index(drop=True, inplace=True)

Ahora que no tenemos valores muy atípicos, identificaremos aquellos outliers más agrupados, que podrían corresponder a un grupo de registros con características similares, pero que difieren un poco del ámbito general de los datos.

In [107]:
df_outliers_lof, lof = find_outliers(data = df.drop(columns = "Attrition"), columns = df.drop(columns = "Attrition").columns, method = "lof")

100%|██████████| 25/25 [00:02<00:00,  9.15it/s]


In [108]:
df_outliers_lof.shape[0]/df.shape[0]

0.03957417264522101

Para estos valores, convertiremos las numéricas en nulos y los imputaremos, de tal forma de que estos outliers no sean tan extremos. Para ello usaremos un random forest. Aunque este proceso sea más lento que otros, en general da mejores resultados.

In [109]:
df.loc[df_outliers_lof.index, num_cols] = np.nan

In [110]:
df_out_impute, out_imputer = impute_nulls(df)

[IterativeImputer] Completing matrix with shape (4321, 38)
[IterativeImputer] Ending imputation round 1/10, elapsed time 23.95
[IterativeImputer] Change: 1.0657451273091891, scaled tolerance: 0.0010000000000000002 
[IterativeImputer] Ending imputation round 2/10, elapsed time 50.45
[IterativeImputer] Change: 0.6552156044092199, scaled tolerance: 0.0010000000000000002 
[IterativeImputer] Ending imputation round 3/10, elapsed time 78.33
[IterativeImputer] Change: 0.6404604734468822, scaled tolerance: 0.0010000000000000002 
[IterativeImputer] Ending imputation round 4/10, elapsed time 104.49
[IterativeImputer] Change: 0.5718270695323939, scaled tolerance: 0.0010000000000000002 
[IterativeImputer] Ending imputation round 5/10, elapsed time 132.18
[IterativeImputer] Change: 0.6076391566105106, scaled tolerance: 0.0010000000000000002 
[IterativeImputer] Ending imputation round 6/10, elapsed time 163.70
[IterativeImputer] Change: 0.6060309185285496, scaled tolerance: 0.0010000000000000002 
[I

In [111]:
df = df_out_impute

In [112]:
df.to_pickle("../datos/prepped.pkl")